```{contents}
```
## **Executor Agent in LangGraph**

An **Executor Agent** in LangGraph is the component responsible for **performing concrete actions** in a multi-agent system.
It converts high-level plans into **real operations**: tool calls, API requests, code execution, data retrieval, and system updates.

Where a **Planner Agent decides *what* to do**, the **Executor Agent decides *how to do it*** and **does it**.

---

### **1. Role in the Agent Architecture**

| Agent                   | Responsibility                           |
| ----------------------- | ---------------------------------------- |
| Planner Agent           | Produces task decomposition and strategy |
| **Executor Agent**      | **Carries out each step of the plan**    |
| Critic / Verifier Agent | Checks correctness & quality             |
| Supervisor Agent        | Orchestrates agents                      |

**Control Flow**

```
User Goal → Planner → Executor → Verifier → Supervisor → Output
```

---

### **2. Why Executor Agents Are Necessary**

LLMs alone generate text.
Real systems must **interact with the world**.

Executor Agents enable:

* Tool invocation
* API interaction
* Code execution
* Database access
* File operations
* Environment manipulation
* Feedback collection

Without an Executor Agent, a system is **only a simulator**.

---

### **3. Executor Agent Core Responsibilities**

| Function           | Description                       |
| ------------------ | --------------------------------- |
| Action Selection   | Choose correct tool / operation   |
| Parameter Binding  | Convert plan into concrete inputs |
| Execution          | Perform the action                |
| State Update       | Write results into shared state   |
| Error Handling     | Retry, fallback, compensate       |
| Progress Reporting | Signal completion or failure      |

---

### **4. Executor Agent in LangGraph — Conceptual Model**

```
[Plan Step]
     ↓
[Executor Agent]
     ↓
[Tool / API / Code]
     ↓
[Observation]
     ↓
[State Update]
```

This loop continues until the plan is complete.

---

### **5. Implementation in LangGraph**

### **State Definition**

```python
class State(TypedDict):
    plan: list[str]
    current_step: int
    observation: str
    done: bool
```

---

### **Executor Node**

```python
def executor(state: State):
    step = state["plan"][state["current_step"]]

    result = tool_registry.run(step)   # real action

    return {
        "observation": result,
        "current_step": state["current_step"] + 1
    }
```

---

### **Planner Node (simplified)**

```python
def planner(state):
    return {"plan": ["search data", "analyze", "summarize"]}
```

---

### **Verifier Node**

```python
def verifier(state):
    if state["current_step"] >= len(state["plan"]):
        return {"done": True}
    return {"done": False}
```

---

### **Graph Wiring**

```python
builder = StateGraph(State)

builder.add_node("planner", planner)
builder.add_node("executor", executor)
builder.add_node("verifier", verifier)

builder.set_entry_point("planner")
builder.add_edge("planner", "executor")
builder.add_edge("executor", "verifier")

builder.add_conditional_edges(
    "verifier",
    lambda s: END if s["done"] else "executor",
    {"executor": "executor", END: END}
)
```

---

### **6. Execution Trace**

```
Plan → Execute Step 1 → Verify → Execute Step 2 → Verify → ... → Done
```

This forms a **cyclic execution loop**.

---

### **7. Executor Agent Variants**

| Variant                 | Use Case                |
| ----------------------- | ----------------------- |
| Single-tool Executor    | Simple workflows        |
| Multi-tool Executor     | Heterogeneous systems   |
| Async Executor          | Long-running operations |
| Parallel Executor       | Map-reduce, batch tasks |
| Human-assisted Executor | Sensitive domains       |
| Autonomous Executor     | Continuous systems      |

---

### **8. Production Enhancements**

| Feature             | Purpose             |
| ------------------- | ------------------- |
| Retries & Backoff   | Reliability         |
| Timeouts            | Deadlock prevention |
| Circuit Breaker     | System protection   |
| Audit Logging       | Compliance          |
| Sandboxing          | Security            |
| Cost Tracking       | Budget control      |
| Human Approval Gate | Risk management     |

---

### **9. Real-World Example Use Cases**

| Domain           | Application                    |
| ---------------- | ------------------------------ |
| Finance          | Data collection, risk modeling |
| Healthcare       | Patient record processing      |
| DevOps           | Infrastructure automation      |
| Research         | Knowledge extraction           |
| Customer Support | Ticket resolution              |

---

### **10. Mental Model**

> **Executor Agent = Operational engine of the system**

It is the **bridge between language and reality**.


### Demonstration

In [1]:
# ===== Executor Agent Demo (Single Cell) =====

from typing import TypedDict, List
from langgraph.graph import StateGraph, END

# ----------------------------
# 1. Shared State Definition
# ----------------------------

class State(TypedDict):
    plan: List[str]
    step: int
    observation: str
    done: bool

# ----------------------------
# 2. Planner Agent
# ----------------------------

def planner(state: State):
    return {
        "plan": ["fetch_data", "process_data", "generate_report"],
        "step": 0,
        "done": False
    }

# ----------------------------
# 3. Tool Registry (Simulated)
# ----------------------------

def run_tool(task: str):
    tools = {
        "fetch_data": "data collected",
        "process_data": "data processed",
        "generate_report": "report created"
    }
    return tools.get(task, "unknown task")

# ----------------------------
# 4. Executor Agent
# ----------------------------

def executor(state: State):
    task = state["plan"][state["step"]]
    result = run_tool(task)

    return {
        "observation": result,
        "step": state["step"] + 1
    }

# ----------------------------
# 5. Verifier Agent
# ----------------------------

def verifier(state: State):
    if state["step"] >= len(state["plan"]):
        return {"done": True}
    return {"done": False}

# ----------------------------
# 6. Graph Construction
# ----------------------------

builder = StateGraph(State)

builder.add_node("planner", planner)
builder.add_node("executor", executor)
builder.add_node("verifier", verifier)

builder.set_entry_point("planner")
builder.add_edge("planner", "executor")
builder.add_edge("executor", "verifier")

builder.add_conditional_edges(
    "verifier",
    lambda s: END if s["done"] else "executor",
    {"executor": "executor", END: END}
)

graph = builder.compile()

# ----------------------------
# 7. Run the Graph
# ----------------------------

final_state = graph.invoke({})
print(final_state)


{'plan': ['fetch_data', 'process_data', 'generate_report'], 'step': 3, 'observation': 'report created', 'done': True}


| Feature               | Planner    | Executor  | Supervisor |
| --------------------- | ---------- | --------- | ---------- |
| Decision-making       | High-level | Low-level | Meta-level |
| Interacts with world  | No         | Yes       | Indirect   |
| Controls other agents | No         | No        | Yes        |
| Manages risk          | No         | Partial   | Yes        |
| Initiates replanning  | Yes        | No        | Yes        |
